In [3]:
import tensorflow as tf
import numpy as np
import pickle
import json
import random
import spacy
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
NEXT_QNT = NEURONS = EPOCHS = 0
with open("../Data/iniV2.pickle", "rb") as f:
    NEXT_QNT , NEURONS, EPOCHS = pickle.load(f)
print(NEURONS,EPOCHS)

64 100


In [5]:
with open("../Data/intents.json","r",encoding='utf-8') as file:
    data = json.load(file)

for i,intent in enumerate(data["intents"]):
    intent["tag"] = "tag"+str(i)

with open("../Data/intents.json","w",encoding='utf-8') as file:
    json.dump(data,file,indent=3,ensure_ascii=False)

In [9]:
nlp = spacy.load('pt_core_news_sm')

In [30]:
words = []
labels = []
docs_x = []
docs_y = []

for intent in data["intents"]:
    for pattern in intent["patterns"]:
        wrds = nlp(pattern.lower())
        wrds = [token.lemma_ for token in wrds if not token.is_stop]
        if wrds == []:
            continue
        words.extend(wrds)
        docs_x.append(wrds)
        docs_y.append(intent["tag"])

    if intent["tag"] not in labels:
        labels.append(intent["tag"])

#from spacy.lang.pt.stop_words import STOP_WORDS

ponctuations = ["?","'",'"',"!",".",","]

words = [w for w in words if w not in ponctuations]
#words = sorted(list(set(words)))
#labels = sorted(labels)
print(len(words))

1012


In [33]:
print(words[0:15])

['oi', 'eae', 'olá', 'dia', 'iai', 'thau', 'ver', 'falar', 'valer', 'util', 'ok', 'agradecer', 'ultimamente', 'tá', 'ir']


In [41]:
training = []
output = []

out_empty = [0 for _ in range(len(labels))]

for x, doc in enumerate(docs_x):
    bag = []
    wrds = []
    doc = nlp(' '.join(doc))
    
    for token in doc:
        if not token.is_stop and token.lemma_ not in ponctuations:
            wrds.append(token.lemma_)
    #wrds = [stemmer.stem(w.lower()) for w in doc]

    for w in words:
        if w in wrds:
            bag.append(1)
        else:
            bag.append(0)

    output_row = out_empty[:]
    output_row[labels.index(docs_y[x])] = 1

    training.append(bag)
    output.append(output_row)


training = np.array(training)
output = np.array(output)
N_CLASSES = len(output[0])
SHAPE = len(training[0])

In [52]:
(len(training[0])+len(output[0]))/2

592.0

In [67]:
#entradas+saídas/2 (opcional).
def dense_layers(inputs):
    x = tf.keras.layers.Dense(592,activation='relu')(inputs)
    #x = tf.keras.layers.BatchNormalization()(x)
    
    #x = tf.keras.layers.Dense(592,activation='relu')(x)
    #x = tf.keras.layers.BatchNormalization()(x)

    x = tf.keras.layers.BatchNormalization()(x)
    #x = tf.keras.layers.Dense(128,activation='relu')(x)
    #x = tf.keras.layers.BatchNormalization()(x)
    
    #x = tf.keras.layers.Dense(256,activation='relu')(x)
    #x = tf.keras.layers.BatchNormalization()(x)
    return x

def classfier_layer(x,N_CLASSES):
    x = tf.keras.layers.Dense(N_CLASSES,activation='softmax',name='classification')(x)
    return x

def final_model(inputs,N_CLASSES):
    dense = dense_layers(inputs)
    
    classfier = classfier_layer(dense,N_CLASSES)
    
    model = tf.keras.Model(inputs=inputs,outputs=classfier)
    
    return model
    
def define_and_compile_model(SHAPE,N_CLASSES):
    inputs = tf.keras.layers.Input(shape=(SHAPE,))
    
    # create the model
    model = final_model(inputs,N_CLASSES)
    
    # compile your model
    model.compile(optimizer='adam',loss='binary_crossentropy',metrics = {'classification' : 'accuracy'})

    return model

In [69]:
model = define_and_compile_model(SHAPE,N_CLASSES)
model.summary()

Model: "model_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         [(None, 1012)]            0         
_________________________________________________________________
dense_14 (Dense)             (None, 592)               599696    
_________________________________________________________________
batch_normalization_2 (Batch (None, 592)               2368      
_________________________________________________________________
classification (Dense)       (None, 172)               101996    
Total params: 704,060
Trainable params: 702,876
Non-trainable params: 1,184
_________________________________________________________________


In [70]:
model.fit(training,output,epochs=50)

Epoch 1/50
17/17 [==============================] - 1s 7ms/step - loss: 0.7419 - accuracy: 0.0536
Epoch 2/50
17/17 [==============================] - 0s 6ms/step - loss: 0.6525 - accuracy: 0.4640
Epoch 3/50
17/17 [==============================] - 0s 6ms/step - loss: 0.5250 - accuracy: 0.6802
Epoch 4/50
17/17 [==============================] - 0s 6ms/step - loss: 0.3333 - accuracy: 0.7597
Epoch 5/50
17/17 [==============================] - 0s 6ms/step - loss: 0.2065 - accuracy: 0.7726
Epoch 6/50
17/17 [==============================] - 0s 6ms/step - loss: 0.1151 - accuracy: 0.7671
Epoch 7/50
17/17 [==============================] - 0s 6ms/step - loss: 0.0636 - accuracy: 0.7523
Epoch 8/50
17/17 [==============================] - 0s 6ms/step - loss: 0.0405 - accuracy: 0.7616
Epoch 9/50
17/17 [==============================] - 0s 6ms/step - loss: 0.0302 - accuracy: 0.7542
Epoch 10/50
17/17 [==============================] - 0s 6ms/step - loss: 0.0266 - accuracy: 0.7745
Epoch 11/50
17/17 [

In [59]:
def bag_of_words(s):
    bag = [0 for _ in range(len(words))]

    ponctuations = ["?","'",'"',"!",".",","]

    doc = nlp(s.lower())
    s_words = []
    for token in doc:
        if not token.is_stop and token.lemma_ not in ponctuations:
            s_words.append(token.lemma_)

    for se in s_words:
        for i, w in enumerate(words):
            if w == se:
                bag[i] = 1
    bag = np.array(bag)
    return bag

In [71]:
while True:
    inp = str(input("Digite: "))
    if inp == "quit":
        break
        
    bag = bag_of_words(inp)
    bag = bag.reshape((1,SHAPE))
    predictions = model.predict(bag)
    results_index = np.argmax(predictions)

    tag = labels[results_index]

    for tg in data['intents']:
        if tg['tag'] == tag:
            responses = tg['responses']
            current_tag = tg

    response = random.choice(responses)
    confidence = predictions[0][results_index]
    print("Model:",response,confidence)

Model: Oi como vai? 0.9888576


In [72]:
model.save("model.h6")

INFO:tensorflow:Assets written to: model.h6/assets


In [73]:
with open("../Data/intents.json","w",encoding='utf-8') as file:
    json.dump(data,file,indent=3,ensure_ascii=False)

In [74]:
with open("../Data/dataV2.pickle", "wb") as f:
    pickle.dump((words, labels, training, output), f)    

In [20]:
with open("../Data/iniV2.pickle", "wb") as f:
    pickle.dump((120, NEURONS, EPOCHS), f)